# Hypothesis to test:
### Removing objective sentences from reviews helps predict star rating from reviews

In [80]:
import numpy as np
import pandas as pd
import pickle
import gzip
import math
import random
from IPython.display import Markdown, display
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor,GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, make_scorer
import statsmodels.api as sm

from utils import rmse, rmse_train_cv, classifier_report, confusion_rmse

# Avoid restarting Kernel
%load_ext autoreload
%autoreload 2

# %autosave 50

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Configuration

In [37]:
# Subsampling from Amazon reviews
NB_SAMPLES = 20000  # up to 200k, then change the input file

## Get user comments

In [38]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path, trunc=0):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1        
    if trunc > 0 and i > trunc: 
        break
  return pd.DataFrame.from_dict(df, orient='index')

In [39]:
data_path = '../../datasets/'
file_name = 'reviews_Movies_and_TV.json.gz'

In [40]:
# comments_df = getDF(data_path + file_name, 200000)
# comments_df.loc[0,'reviewText']
# print(comments_df.shape)
# comments_df.head(1)

In [41]:
# Save the records
# import pickle
# pickle_out = open(data_path + "amzn_200k.pickle","wb")
# pickle.dump(comments_df, pickle_out)
# pickle_out.close()

In [42]:
# Reads records
# import pickle
# pickle_in = open(data_path + "amzn_200k.pickle","rb")
# comments_df = pickle.load(pickle_in)

## Subsample

In [43]:
# small = comments_df.loc[:NB_SAMPLES, :]
# print(len(small.index))
# small.head(3)

In [44]:
pickle_in = open('pickles/five_k_balanced_reviews.pkl',"rb")
balanced_reviews = pickle.load(pickle_in)
pickle_in.close;

In [45]:
balanced_reviews.iloc[random.sample(range(25000), 2), :]

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
3629,A2Y70AOI3NG37J,0310263662,"S. Koropeckyj ""Romi Panchir""","[8, 25]",As so many other people on this review page ha...,2.0,Path to heaven...,1094083200,"09 2, 2004"
31932,A3V3W7F8IMNIO7,0767815963,Kat,"[3, 3]","Like everyone else, I've been searching for th...",4.0,There's hope out there!!,1223942400,"10 14, 2008"


## Star rating modeling for base case, based on tf-idf

In [46]:
X_train, X_cv, y_train, y_cv = train_test_split(
  balanced_reviews['reviewText'], balanced_reviews['overall'], test_size=0.2, random_state=0)
X_train.shape

(20000,)

### TF-IDF setup

In [82]:
MAX_FEATURES = 20000

In [83]:
tfidf = TfidfVectorizer(lowercase=True, 
                        stop_words='english', 
                        max_features=MAX_FEATURES,
                        norm='l2',            # normalize each review
                        use_idf=True)        # Keep high weight for most common words

In [86]:
X_train_tf = tfidf.fit_transform(X_train)
print(len(tfidf.vocabulary_))
X_cv_tf = tfidf.transform(X_cv)
print(X_cv_tf.shape)

20000
(5000, 20000)


### Gradient Boosting Classifier

In [50]:
# Gradient Boosting Classifier parameters
N_TREES = 300 # math.floor(np.sqrt(NB_SAMPLES) * 1.2)
LEARN_RATE = 0.1
MAX_DEPTH = 15
MIN_IN_LEAF = 2 #7

In [51]:
gbc = GradientBoostingClassifier(learning_rate=LEARN_RATE, 
                                n_estimators=N_TREES, 
                                min_samples_leaf=MIN_IN_LEAF,
                                max_depth=MAX_DEPTH,
                                random_state=0)

In [53]:
if False:
    gb_pipe = Pipeline([('vect', tfidf), ('gb', gbc)])
    gb_pipe.fit(X_train, y_train)
    pickle.dump(gb_pipe, open('pickles/GBC_balanced_comments_'
                           + str(N_TREES) + '_trees_' 
                           + str(LEARN_RATE) + '_lr_' 
                           + str(MAX_DEPTH) + '_maxdpth_'
                           + str(MIN_IN_LEAF) + '_minleaf_'
                           + str(MAX_FEATURES) + '_feats_'
                           + '.pkl', 'wb'))
else:
    pickle_in = open("pickles/GBC_balanced_comments_300_trees_0.1_lr_15_maxdpth_2_minleaf_20000_feats_.pkl",
                     "rb")
    gb_pipe = pickle.load(pickle_in)

In [72]:
%reload_ext autoreload
print(MAX_FEATURES, ' features', N_TREES,'trees; ',
      LEARN_RATE,'learn_rate; ', MAX_DEPTH, 'max_dpth; ',
      MIN_IN_LEAF, 'min_in_leaf')
classifier_report(gb_pipe, X_train, y_train,
                  'Gradient Boosting Classifier on training set')
classifier_report(gb_pipe, X_cv, y_cv, 
                  'Gradient Boosting Classifier on CV set')

20000  features 300 trees;  0.1 learn_rate;  15 max_dpth;  2 min_in_leaf


### Report for Gradient Boosting Classifier on training set:

##### Confusion RMSE: 0.1

#### Confusion Matrix:

[[4017    0    2    0    0]
 [   1 3964    8    4    3]
 [   8    4 3995   10   16]
 [   1    2    3 3972    4]
 [   0    0    3    0 3983]]


...

##### Confusion RMSE: 1.3

#### Confusion Matrix:

[[513 230 123  63  52]
 [224 339 270 121  66]
 [ 94 199 339 216 119]
 [ 62 110 282 326 238]
 [ 45  56 144 233 536]]


#### Classification Report:

              precision    recall  f1-score   support

         1.0       0.55      0.52      0.53       981
         2.0       0.36      0.33      0.35      1020
         3.0       0.29      0.35      0.32       967
         4.0       0.34      0.32      0.33      1018
         5.0       0.53      0.53      0.53      1014

   micro avg       0.41      0.41      0.41      5000
   macro avg       0.41      0.41      0.41      5000
weighted avg       0.41      0.41      0.41      5000



## Grid search

In [93]:
if False:
    grid = {
        'learning_rate': [.07, .01],
        'max_depth': [6, 12],
        'min_samples_leaf': [2, 5],
    #     'max_features': ['sqrt', None],
        'n_estimators': [300],
        'random_state': [0]
    }
else:
    grid = {
    'learning_rate': [1],
    'max_depth': [2], 6, 12],
    'min_samples_leaf': [2, 5],
#     'max_features': ['sqrt', None],
    'n_estimators': [300],
    'random_state': [0]
}
    
confusion_score = make_scorer(confusion_rmse, greater_is_better=False)

gbc_grid_cv = GridSearchCV(
    GradientBoostingClassifier(), 
    grid,
    cv=2,  # number of folds
    scoring=confusion_score,
    return_train_score=True,
    verbose=1, 
    n_jobs=-1)
gbc_grid_cv.fit(X_train_tf, y_train)

Fitting 2 folds for each of 8 candidates, totalling 16 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


KeyboardInterrupt: 

In [ ]:
y_pred = gbc_grid_cv.predict(X_cv_tf)

In [ ]:
print(gbc_grid_cv.best_params_)
print(gbc_grid_cv.best_score_)
res_df = pd.DataFrame(gbc_grid_cv.cv_results_)
res_df

In [ ]:
pickle_out = open("pickles/GBC_gridSearch_results.pkl","wb")
pickle.dump(res_df, pickle_out)
pickle_out.close()

### Logistic regressor

In [14]:
lr = LogisticRegression(random_state=0, solver='lbfgs',
                       multi_class='multinomial',max_iter=5000)

In [15]:
lr_pipe = Pipeline([('vect', tfidf), ('lr', lr)])
lr_pipe.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=20000, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,..., penalty='l2', random_state=0, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False))])

In [18]:
confusion_mat = confusion_matrix(y_cv,lr_pipe.predict(X_cv))
type(confusion_mat)

numpy.ndarray

In [24]:
confusion_mat

array([[576, 235,  85,  45,  40],
       [225, 409, 235,  94,  57],
       [ 94, 206, 349, 203, 115],
       [ 50,  93, 235, 377, 263],
       [ 43,  33,  89, 235, 614]])

In [35]:
%reload_ext autoreload
res = confusion_rmse(confusion_mat)
res

7110.0


84.32081593533118

In [27]:
4 * 115

460

In [46]:
%reload_ext autoreload
classifier_report(lr_pipe, X_train, y_train, 'Logistic regressor on training set')
classifier_report(lr_pipe, X_cv, y_cv, 'Logistic regressor on CV set')

### Report for Logistic regressor on training set:

#### Classification Report:

              precision    recall  f1-score   support

         1.0       0.76      0.82      0.79      4019
         2.0       0.75      0.72      0.73      3980
         3.0       0.73      0.68      0.71      4033
         4.0       0.74      0.70      0.72      3982
         5.0       0.75      0.82      0.78      3986

   micro avg       0.75      0.75      0.75     20000
   macro avg       0.75      0.75      0.75     20000
weighted avg       0.75      0.75      0.75     20000



#### Confusion Matrix:

[[3308  375  143   87  106]
 [ 541 2846  317  151  125]
 [ 280  339 2761  364  289]
 [ 113  154  370 2801  544]
 [ 101   83  177  375 3250]]


### Report for Logistic regressor on CV set:

#### Classification Report:

              precision    recall  f1-score   support

         1.0       0.58      0.59      0.59       981
         2.0       0.42      0.40      0.41      1020
         3.0       0.35      0.36      0.36       967
         4.0       0.40      0.37      0.38      1018
         5.0       0.56      0.61      0.58      1014

   micro avg       0.47      0.47      0.47      5000
   macro avg       0.46      0.46      0.46      5000
weighted avg       0.46      0.47      0.46      5000



#### Confusion Matrix:

[[576 235  85  45  40]
 [225 409 235  94  57]
 [ 94 206 349 203 115]
 [ 50  93 235 377 263]
 [ 43  33  89 235 614]]


### Stochastic gradient descent classifier

In [120]:

sgd= SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, 
                                        random_state=0, max_iter=500, tol=None)
sgd.fit(X_train, y_train)

y_cv_pred = sgd.predict(X_cv)

In [119]:
print(classification_report(y_cv, y_cv_pred))
print(confusion_matrix(y_cv,y_cv_pred))

              precision    recall  f1-score   support

         1.0       0.50      0.76      0.60       981
         2.0       0.42      0.22      0.29      1020
         3.0       0.39      0.31      0.34       967
         4.0       0.40      0.25      0.31      1018
         5.0       0.51      0.77      0.61      1014

   micro avg       0.46      0.46      0.46      5000
   macro avg       0.44      0.46      0.43      5000
weighted avg       0.44      0.46      0.43      5000

[[749  90  57  26  59]
 [430 229 191  80  90]
 [162 141 295 181 188]
 [ 97  70 175 257 419]
 [ 70  14  48 101 781]]


## Split comments into separate sentences

In [9]:
from nltk.tokenize import sent_tokenize
small['sentence'] = small['reviewText'].map(sent_tokenize)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [10]:
small.columns

Index(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText',
       'overall', 'summary', 'unixReviewTime', 'reviewTime', 'sentence'],
      dtype='object')

In [11]:
small.drop(['reviewerName', 'helpful', 'reviewText', 'summary', 
            'unixReviewTime', 'reviewTime'], axis=1, inplace=True)

/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [12]:
small.shape

(2001, 4)

In [13]:
sentences = small['sentence'] \
.apply(pd.Series) \
.merge(small, left_index = True, right_index = True) \
.drop(['sentence'], axis = 1) \
.melt(id_vars = ['reviewerID', 'asin','overall'], value_name = 'sentence') \
.drop(['variable'], axis = 1) \
.dropna()

print(sentences.shape)
sentences.head(3)

(11115, 4)


,reviewerID,asin,overall,sentence
0,A3R5OBKS7OM2IR,0000143502,5.0,This has some great tips as always and is help...
1,A3R5OBKS7OM2IR,0000143529,5.0,This is a great pastry guide.
2,AH3QC2PC1VTGP,0000143561,2.0,I have to admit that I am a fan of Giada's coo...


## Sentence-level prep & cleaning

In [14]:
%reload_ext autoreload
from utils import split_n_lower, not_about_support

### Split into words and lower the case

In [15]:
sentences['words'] = sentences['sentence'].apply(lambda s: split_n_lower(s))

In [16]:
print(sentences.shape)
sentences.head(3)

(11115, 5)


,reviewerID,asin,overall,sentence,words
0,A3R5OBKS7OM2IR,0000143502,5.0,This has some great tips as always and is help...,"[this, has, some, great, tips, as, always, and..."
1,A3R5OBKS7OM2IR,0000143529,5.0,This is a great pastry guide.,"[this, is, a, great, pastry, guide, .]"
2,AH3QC2PC1VTGP,0000143561,2.0,I have to admit that I am a fan of Giada's coo...,"[i, have, to, admit, that, i, am, a, fan, of, ..."


### Keep support-related sentences as they probably have impact on rating

In [17]:
# on_movies_filter = [not_about_support(word) for word in sentences['words']]
sentences_on_movie = sentences #[on_movies_filter]

print('Removing {} records'.format(sentences.shape[0]- sentences_on_movie.shape[0]))

Removing 0 records


In [18]:
sentences_on_movie.shape

(11115, 5)

### Base case: A reviews with objective and subjective sentences

In [19]:
# Kernel dies here at 50K samples
all_reviews_groups = sentences_on_movie.groupby(['reviewerID','asin'])


In [20]:
all_reviews_stars = all_reviews_groups['overall'].mean()
all_reviews_stars[:3]

reviewerID      asin      
A100CQXJ6D44T9  0005119367    5.0
A100Z2S0880G9A  0005019281    5.0
A102Z4PIK7CYD8  030714139X    5.0
Name: overall, dtype: float64

In [21]:
all_reviews_comments = all_reviews_groups['words'].sum()
print(sentences_on_movie.iloc[0, 4])
print(all_reviews_comments.shape)
print(all_reviews_comments[0])
len(all_reviews_comments)

['this', 'has', 'some', 'great', 'tips', 'as', 'always', 'and', 'is', 'helping', 'me', 'to', 'complete', 'my', 'good', 'eats', 'collection', '.']
(2000,)
['awesome', 'movie', 'and', 'great', 'story', '.', 'highly', 'recommend', 'this', 'movie', 'and', 'follows', 'closely', 'to', 'biblical', 'accounts', '.', 'the', 'message', 'is', 'strong', 'about', 'forgiveness', 'and', 'faithfulness', '.']


2000

### Remove objective sentences for case B using obj-subj model

In [22]:
# pd.set_option('display.max_colwidth', -1)
# sentences_on_movie['sentence']
sentences_on_movie.shape

(11115, 5)

In [23]:
sentences_on_movie.head(2)

,reviewerID,asin,overall,sentence,words
0,A3R5OBKS7OM2IR,0000143502,5.0,This has some great tips as always and is help...,"[this, has, some, great, tips, as, always, and..."
1,A3R5OBKS7OM2IR,0000143529,5.0,This is a great pastry guide.,"[this, is, a, great, pastry, guide, .]"


#### Vectorize along the word space of the obj-subj training set

In [25]:
tfidf = pickle.load(open('pickles/Obj-Subj_tfidf.pkl', 'rb'))
len(tfidf.vocabulary_)

20893

In [35]:
sentences_tfidf = tfidf.transform(sentences_on_movie['sentence']).todense()

In [36]:
sentences_tfidf.shape

(11115, 20893)

#### Apply the obj-subj model

In [29]:
N_TREES = 100
LEARN_RATE = 0.1
MIN_IN_LEAF = 10
pickle_in = open('pickles/GBC_'+ str(N_TREES) +'_' + str(LEARN_RATE) 
                        +'_' + str(MIN_IN_LEAF) + '_20min.pkl', 'rb')
gb_model = pickle.load(pickle_in)

In [30]:
y_test = gb_model.predict(sentences_tfidf)
len(y_test)

11115

In [ ]:
subjective_sentences = sentences_on_movie[y_test == 1]

In [34]:
display(Markdown('### Removing {} objective sentences'
                 .format(len(y_test) - len(subjective_sentences))))

### Removing 3340 objective sentences

In [ ]:
subjective_sentences.head(2)

#### Merge the sentences back into paragraph reviews

In [ ]:
subj_groups = subjective_sentences.groupby(['reviewerID','asin'])
subj_reviews_stars = subj_groups['overall'].mean()
# subjective_reviewssubjective_reviews['sentence'].apply(lambda x: x.sum())
# subjective_reviews_reviews = 
subj_reviews_stars[:3]

In [ ]:
subj_review_comments = subj_groups['words'].sum()
print(subj_review_comments.shape)
print(subj_review_comments[0])
subj_review_comments[:3]

### Check that stars still correspond to the right movie

In [ ]:
start = 6000
end = 6010
all_reviews_comments.loc[('A33Z7JTV7SSW9Y', '0718000315')]


In [ ]:
print(all_reviews_stars.loc[('A33Z7JTV7SSW9Y', '0718000315')])
print(sentences_on_movie.loc[sentences_on_movie['reviewerID']=='A33Z7JTV7SSW9Y']) 
# and sentences_on_movie['asin']=='0718000315'])

In [ ]:
pd.options.display.max_colwidth = -1
print(small.loc[small['reviewerID']=='A33Z7JTV7SSW9Y'])

In [ ]:
sentences_on_movie[start:end]

## Create emotion vectors

In [ ]:
print('Total number of reviews:', all_reviews_comments.shape[0])
print('Total number of subjective reviews:', subj_review_comments.shape[0])

In [ ]:
%reload_ext autoreload
from emotions_seven import Emotions7
emote = Emotions7()

In [ ]:
all_reviews_emotions = emote.vectorize(all_reviews_comments)
print(all_reviews_emotions.shape)

In [ ]:
# emote.emotions_in_text

In [ ]:
all_reviews_emotions.shape

In [ ]:
# all_revs_with_emotions = all_reviews_emotions[emote.emotions_in_text == True]

In [ ]:
# print(all_revs_with_emotions.shape)
# all_revs_stars = all_reviews_stars[emote.emotions_in_text]
all_reviews_emotions[0]

In [ ]:
subj_reviews_emotions = emote.vectorize(subj_review_comments)
print(subj_reviews_emotions.shape)
subj_reviews_emotions[0]

## Fit a model on base case (all comments) for star rating prediction

In [ ]:
X_subj_train, X_subj_cv, y_subj_train, y_subj_cv = train_test_split(
    subj_reviews_emotions, subj_reviews_stars, test_size=0.2, random_state=0)
X_subj_train.shape

In [ ]:
gbc_subj = GradientBoostingClassifier(learning_rate=LEARN_RATE, 
                                n_estimators=N_TREES, 
                                min_samples_leaf=MIN_IN_LEAF,
                                random_state=0)
gbc_subj.fit(X_subj_train, y_subj_train)

In [ ]:
print('Gradient Boosting Classifier')
print('Training score using all comments: {0:.2f}'
      .format(gbc_all.score(X_train, y_train)))
print('CV score using all comments: {0:.2f}'
      .format(gbc_all.score(X_cv, y_cv)))
print('')

# print('Training score using subjective comments only: {0:.2f}'
#       .format(gbc_subj.score(X_subj_train, y_subj_train)))
# print('CV score using subjective comments only: {0:.2f}'
#       .format(gbc_subj.score(X_subj_cv, y_subj_cv)))


## Other techniques

In [ ]:
# from sklearn.linear_model import LogisticRegression
# lr = LogisticRegression(random_state=0, solver='lbfgs',
#                        multi_class='multinomial',max_iter=1000)
# lr.fit(X_subj_train, y_subj_train)
# print(lr.score(X_subj_train, y_subj_train))
# print(lr.score(X_subj_cv, y_subj_cv))

In [ ]:
ols_all = sm.OLS(y_train, X_train)
results_all = ols_all.fit()
results_all.summary()

In [ ]:
ols_subj = sm.OLS(y_subj_train, X_subj_train)
results_subj = ols_subj.fit()
results_subj.summary()

In [ ]:
import seaborn as sns

all_reviews_emotions, all_reviews_stars

sns.heatmap(raw_df.corr(), annot=True)